# Extract relationshipts between concepts
Currently not using this

In [2]:
import ollama.client as client
client.list()

[{'name': 'mistral:instruct',
  'modified_at': '2023-10-14T11:39:58.684144971+05:30',
  'size': 4108916688,
  'digest': '8aa307f73b2622af521e8f22d46e4b777123c4df91898dcb2e4079dc8fdf579e'},
 {'name': 'mistral-openorca:latest',
  'modified_at': '2023-10-14T14:27:09.318837452+05:30',
  'size': 4108928254,
  'digest': '12dc6acc14d0c4d3309e035cac6ff83e64c023aa774b21c41e342982699f436c'}]

In [87]:
mistral_instruct = 'mistral:instruct'
mistral_openorca = 'mistral-openorca:latest'
mcard = client.show(mistral_openorca)
mcard['template']

'{{- if .System }}\n<|im_start|>system {{ .System }}<|im_end|>\n{{- end }}\n<|im_start|>user\n{{ .Prompt }}<|im_end|>\n<|im_start|>assistant\n'

In [86]:
system_prompt = """
Your task is to find find a relationships between the entities given as a python list.\n
You can use the provided context (delimited between ```) and no other prior knowledge. \n
Follow this chain of thought
Thought 1: What are the two entities as per the context. 
Thought 2: What is the relationship between the given entities as per the given context?. 
Thought 3: Is this a direct relationship as per the context.
Format your response as JSON with the following structure:\n
{ 
    "Entity 1": First Entity,
    "Entity 2": Second Entity,
    "Relationship": Explain the Relationship,
    "Related": 'Yes' only if a direct relation can be infered, otherwise 'No'
}
"""
prompt = """
Entites: ['Cryosurgery', 'non-narcotic pain'].

Context:  ``` 
The two primary types of cancer surgery are open surgery and minimally invasive surgery.

In open surgery, the surgical oncologist makes a large incision, usually to remove all or part of a tumor and some of the surrounding healthy tissue (margins).

Minimally invasive surgical techniques may involve the techniques listed below.

Laparoscopy: A surgical oncologist makes a few small incisions and inserts a laparoscope —a thin tube with a tiny camera attached to it—into one of the incisions to capture an interior image, while inserting surgical tools into the other incisions to excise malignancies and surrounding tissue.

Laser surgery: The surgeon uses a narrow beam of high-intensity light to remove a tumor.

Cryosurgery: The surgeon uses liquid nitrogen to freeze and kill cancer cells.

Robotic surgery: This surgery is similar to a laparoscopic surgery. However, instead of manipulating surgical tools by hand, the surgeon uses a computer console to operate the robotic tools.

Other procedures, such as endoscopies, embolization, Mohs micrographic surgery and pleuroscopies, may be performed by non-surgeons, including dermatologists, radiation oncologists and interventional pulmonologists, depending on the procedure.

Non-surgical treatments may take place before surgery (neoadjuvant therapy) or after surgery (adjuvant therapy) to help prevent cancer growth, metastasis or recurrence. The treatments may include chemotherapy, radiation therapy or hormone therapy.

The Advanced Surgical Recovery Program (ASURE) at City of Hope is designed to help patients recover from surgery more quickly and with fewer complications. ASURE is intended to improve surgical outcomes and enhance the patient experience before, during and after surgery, while also reducing the length of a patient’s hospital stay.

The program is delivered by City of Hope multidisciplinary teams, which include surgeons, anesthesiologists, nutritionists, behavioral health therapists, nurses and other clinicians. They work collaboratively to implement evidenced-informed protocols, all under one roof. ASURE combines post-surgical rehabilitation with pre-habilitation, by preparing patients before surgery.

Typically, ASURE protocols involve nutrition, pain management with non-narcotic pain relievers and physical activity within 24 hours after surgery. The use of enhanced recovery methods like those used in ASURE has been shown to reduce fatigue, post-surgery complications and the amount of narcotics used in post-surgical care, while helping patients resume their everyday activities more quickly.
```
"""

combined_prompt = system_prompt + "\n" + prompt
response, context = client.generate(
    model_name=mistral_openorca, prompt=prompt, system=system_prompt
)

 {
    "Entity 1": Cryosurgery,
    "Entity 2": non-narcotic pain,
    "Relationship": Cryosurgery uses liquid nitrogen to freeze and kill cancer cells, which is a part of the minimally invasive surgical techniques. Non-narcotic pain is one of the methods used in post-surgical care for pain management as per ASURE protocols.
    "Related": 'Yes'
}